---
title: "Pertemuan 5 : Time Series Visualization"
image: static\images\timeseries.png
description: "Handling time series data"
date: 05/07/2024
page-navigation: true
jupyter: python3
format: html
---

Kembali ke [EDA](./eda2024genap.qmd)

# `datetime` object

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

df = pd.read_csv('https://raw.githubusercontent.com/farhanage/dataset-for-study/main/Electric_Production.csv', names=['date', 'electric_production'], header=0)

df.head(15)

In [ ]:
# Mengecek tipe data tiap kolom
df.info()

Perhatikan bahwa kolom date memiliki tipe data `object` (string). Pada pertemuan ini, kita akan membahas suatu tipe data baru yang bernama `datetime` yang digunakan untuk memanipulasi data runtun waktu (time series).

## Converting `object` to `datetime`

Perhatikan code cell berikut untuk mengubah data bertipe `object` menjadi `datetime`

In [ ]:
# Mengubah data `object` -> `datetime`
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y')

:::{.callout-note}
# Date Format

Argumen `format='%m/%d/%Y'` digunakan untuk membaca format penanggalan yang tertulis pada kolom yang ingin kita ubah. `%d` menandakan hari, `%m` untuk bulan, `%y` untuk tahun 2 digit dan `%Y` untuk tahun 4 digit.

```
e.g. : '2024-05-26' -> '%Y-%m-%d'
```
:::

In [ ]:
df.info()

Terlihat bahwa kolom `date` kini memiliki tipe data `datetime64[ns]`.

In [ ]:
df.head()

Terlihat setelah dilakukan perubahan tipe data, format penanggalan pada kolom `date` juga berubah.

# Apa perbedaan `string` dan `datetime`?

Object `datetime` memiliki attribut tanggal, bulan, waktu, hari, jam, menit, bahkan detik suatu observasi yang dapat kita akses melalui library datetime.

In [ ]:
# Ambil tahun dari setiap observasi
df['date'].dt.year

In [ ]:
# Ambil Bulan dari setiap observasi
df['date'].dt.month

In [ ]:
# Ambil Tanggal dari setiap observasi
df['date'].dt.day

object `datetime` dapat digunakan sebagai nilai numerik dalam visualisasi data

In [ ]:
sns.lineplot(x='date', y='electric_production', data=df)

plt.title('Jumlah produksi listrik per tahun (1985-2018)')

plt.show()

# Grouping

Bagaimana cara kita memanfaatkan attribut-attribut datetime? Gunakan grouping method (`groupby`) untuk mendapatkan insight baru dari data per satuan waktu tertentu.

**Informasi jumlah total produksi energi per tahunnya**

In [ ]:
df['electric_production'].groupby(df['date'].dt.year).sum() # sum of electric_production, grouped by year

**Informasi rata-rata produksi energi setiap bulan selama 33 tahun terakhir**

In [ ]:
df['electric_production'].groupby(df['date'].dt.month).mean() # mean of electric_production, grouped by month

## Plots

In [ ]:
sum_per_year = df['electric_production'].groupby(df['date'].dt.year).sum() # sum of electric_production, grouped by year

sns.lineplot(sum_per_year)

plt.title('Jumlah produksi listrik per tahun (1985-2018)')

plt.show()

kenapa tahun 2018 turun drastis? cek semua observasi pada tahun 2018

In [ ]:
# Ambil data dengan tahun == 2018
df[df['date'].dt.year == 2018]

Ternyata tahun 2018 hanya memiliki 1 observasi, sehingga tidak dapat dibandingkan dengan jumlah produksi tahun-tahun sebelumnya.

In [ ]:
# buang data tahun 2018
df.drop(df[df['date'].dt.year == 2018].index, axis=0, inplace=True)

In [ ]:
df[df['date'].dt.year == 2018]

Data sudah berhasil dibuang, sehingga dapat dilakukan visualisasi yang lebih akurat

In [ ]:
sum_per_year = df['electric_production'].groupby(df['date'].dt.year).sum() # sum of electric_production, grouped by year

sns.lineplot(sum_per_year)

plt.xlabel('year')

plt.ylabel('electric production')

plt.title('Jumlah produksi listrik per tahun (1985-2017)')

plt.show()

Interpretasinya?

In [ ]:
avg_per_month = df['electric_production'].groupby(df['date'].dt.month).mean() # mean of electric_production, grouped by month

sns.barplot(avg_per_month)

plt.xlabel('month')

plt.ylabel('average electric production')

plt.title('Rata-rata produksi listrik per bulan (1985-2017)')

plt.show()

Interpretasi? Kaitkan dengan musim?

# Case Study

Download dataset di sini : [Energy Time Series](https://drive.google.com/drive/folders/122UC_72nX8DocmqdmNfBIPXMmk3zr_EP?usp=sharing)

Metadata :

1. energy_reading.csv
- `building_id` - Foreign key for the building metadata.
- `meter` - The meter id code. Read as {0: electricity, 1: chilledwater, 2: steam, 3: hotwater}. Not every building has all meter types.
- `timestamp` - When the measurement was taken
- `meter_reading` - The target variable. Energy consumption in kWh (or equivalent). Note that this is real data with measurement error, which we expect will impose a baseline level of modeling error. UPDATE: as discussed here, the site 0 electric meter readings are in kBTU.

2. building_metadata.csv
- `site_id` - Foreign key for the weather files.
- `building_id` - Foreign key for training.csv
- `primary_use` - Indicator of the primary category of activities for the building based on EnergyStar property type definitions
- `square_feet` - Gross floor area of the building
- `year_built` - Year building was opened
- `floor_count` - Number of floors of the building

3. weather.csv
Weather data from a meteorological station as close as possible to the site.
- `site_id`
- `air_temperature` - Degrees Celsius
- `cloud_coverage` - Portion of the sky covered in clouds, in oktas
- `dew_temperature` - Degrees Celsius
- `precip_depth_1_hr` - Millimeters
- `sea_level_pressure` - Millibar/hectopascals
- `wind_direction` - Compass direction (0-360)
- `wind_speed` - Meters per second

Challenge : Gali informasi sebanyak-banyaknya dari data tersebut